<a href="https://colab.research.google.com/github/mphill82/Coursera_Capstone/blob/main/Clustering_Toronto_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering Toronto Neighborhoods by Venues

In [29]:
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup

### Part 1 - Acquiring Neighborhood information from Wikipedia

First we scrape data from wikipedia on neighborhoods and boroughs of each postal code in Toronto.

In [2]:
page=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(page,'html.parser')

In [4]:
table = soup.find('table')

We extract the postal code, borough, and list of associated neighborhoods from each cell of the table on wikipedia and store it to dataframe df.

In [28]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]   #first three characters in a row
        cell['Borough'] = (row.span.text).split('(')[0]   #everything in the span section of a row before the first
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)   #starting after the first parentheses, remove the rest of the parentheses and replace forward slashes with commas

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

This is what the dataframe looks like.

In [30]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


The dataframe has 103 entry rows and 3 feature columns.

In [25]:
df.shape

(103, 3)

### Part 2 - Acquiring Latitude and Longitude data

### Part 3 - Clustering with Foursquare Venue data